# CrewAI example connecting to Sap Generative AI Hub via LiteLLM

## [How CrewAI works](https://docs.crewai.com/en/introduction)

## Installation


%pip install crewai litellm

## Credentials for SAP Gen AI Hub
Get the service key from your SAP BTP tenant with AI subscription.

Add the following variables from the service key in a file called ".env" and put it in the same folder where you run the notebook:
```
AICORE_AUTH_URL="https://* * * .authentication.sap.hana.ondemand.com/oauth/token"
AICORE_CLIENT_ID=" *** "
AICORE_CLIENT_SECRET=" *** "
AICORE_RESOURCE_GROUP=" *** "
AICORE_BASE_URL="https://api.ai.***.cfapps.sap.hana.ondemand.com/
```

## Run the CrewAI with LiteLLM and SAP LLMs

In [ ]:
from crewai import Agent, Crew, Task
from crewai.tools import tool
from dotenv import load_dotenv

Load your credentials as environment variables that Litellm can use automatically.

In [ ]:
load_dotenv()

In [ ]:
from dotenv import load_dotenv
from litellm import completion

load_dotenv()

response = completion(
  model="sap/gpt-4o",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(response)

Define the agent tools.

In [ ]:
@tool("get_weather")
def get_weather(city: str) -> str:
    """Mock function"""
    city_normalized = city.lower().replace(" ", "")

    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return f"The weather in {city} is sunny with a temperature of 20°C."

User can select a city.

In [ ]:
city = "london"

Define the Agent and the SAP LLM to be used by CrewAI.

In [ ]:
agent = Agent(
    role="Weather presenter",
    goal=f"Prepare a couple of sentences in TV speach about weather in the {city}, "
         f"using information from the get_weather tool",
    backstory="You are the weather presenter on TV.",
    llm="sap/gpt-4o",
    tools=[get_weather],
    allow_delegation=False,
)

Define tasks for the agents.

In [ ]:
agent_task = Task(
    description=(
        f"Write a couple of sentences for TV weather report in {city} including a small joke."
    ),
    expected_output=(
        "Good quality text of two sentences about weather with small joke."
    ),
    agent=agent,
)

Create the crew with the Agent and task.

In [ ]:
crew = Crew(
    agents=[agent],
    tasks=[agent_task],
    verbose=True,
)

Run the crew

In [ ]:
result = crew.kickoff()
print("\n📘 Result:\n", result)